

Table of Contents:

1. Term Frequency (TF)
2. Inverse Document Frequency (IDF)
3. TF * IDF
4. Vector Space Models and Representation – Cosine Similarity




Let us imagine that you are doing a search on below documents with the following query: **life learning**

* **Document 1** : I want to start learning to charge something in life.
* **Document 2** : learning something about me no one else knows
* **Document 3** : Never stop learning

The query is a free text query. It means a query in which the terms of the query are typed freeform into the search interface, without any connecting search operators.

Let us go over each step in detail to see how it all works.

# 1.Term Frequency(TF)
Term Frequency also known as TF measures the number of times a term (word) occurs in a document. Given below is the code and the terms and their frequency on each of the document.


In [1]:
import math
import pandas as pd
import numpy as np

In [2]:
#documents
doc1 = "I want to start learning to charge something in life"
doc2 = "reading something about life no one else knows"
doc3 = "Never stop learning"
#query string
query = "life learning"

**NOTE :** Text Preprocessing Steps are ignored as the objective of this kernel is to explain and develop TF-IDF and cosine similarity from scratch

In [3]:
#term -frequenvy :word occurences in a document
def compute_tf(docs_list):
    for doc in docs_list:
        doc1_lst = doc.split(" ")
        wordDict_1= dict.fromkeys(set(doc1_lst), 0)

        for token in doc1_lst:
            wordDict_1[token] +=  1
        df = pd.DataFrame([wordDict_1])
        idx = 0
        new_col = ["Term Frequency"]    
        df.insert(loc=idx, column='Document', value=new_col)
        print(df)
        
compute_tf([doc1, doc2, doc3])

         Document  life  charge  something  learning  start  in  to  want  I
0  Term Frequency     1       1          1         1      1   1   2     1  1
         Document  life  else  something  no  one  knows  about  reading
0  Term Frequency     1     1          1   1    1      1      1        1
         Document  stop  Never  learning
0  Term Frequency     1      1         1


* In reality each document will be of different size. On a large document the frequency of the terms will be much higher than the smaller ones. Hence we need to normalize the document based on its size. 
* A simple trick is to divide the term frequency by the total number of terms. 
* For example in Document 1 the term game occurs two times. The total number of terms in the document is 10. Hence the normalized term frequency is 2 / 10 = 0.2. 


Given below are the normalized term frequency for all the documents.

In [4]:
#Normalized Term Frequency
def termFrequency(term, document):
    normalizeDocument = document.lower().split()
    return normalizeDocument.count(term.lower()) / float(len(normalizeDocument))

def compute_normalizedtf(documents):
    tf_doc = []
    for txt in documents:
        sentence = txt.split()
        norm_tf= dict.fromkeys(set(sentence), 0)
        for word in sentence:
            norm_tf[word] = termFrequency(word, txt)
        tf_doc.append(norm_tf)
        df = pd.DataFrame([norm_tf])
        idx = 0
        new_col = ["Normalized TF"]    
        df.insert(loc=idx, column='Document', value=new_col)
        print(df)
    return tf_doc

tf_doc = compute_normalizedtf([doc1, doc2, doc3])
tf_doc

        Document  life  charge  something  learning  start   in   to  want  \
0  Normalized TF   0.1     0.1        0.1       0.1    0.1  0.1  0.2   0.1   

     I  
0  0.1  
        Document   life   else  something     no    one  knows  about  reading
0  Normalized TF  0.125  0.125      0.125  0.125  0.125  0.125  0.125    0.125
        Document      stop     Never  learning
0  Normalized TF  0.333333  0.333333  0.333333


[{'life': 0.1,
  'charge': 0.1,
  'something': 0.1,
  'learning': 0.1,
  'start': 0.1,
  'in': 0.1,
  'to': 0.2,
  'want': 0.1,
  'I': 0.1},
 {'life': 0.125,
  'else': 0.125,
  'something': 0.125,
  'no': 0.125,
  'one': 0.125,
  'knows': 0.125,
  'about': 0.125,
  'reading': 0.125},
 {'stop': 0.3333333333333333,
  'Never': 0.3333333333333333,
  'learning': 0.3333333333333333}]

# 2. Inverse Document Frequency (IDF)





In [5]:
def inverseDocumentFrequency(term, allDocuments):
    numDocumentsWithThisTerm = 0
    for doc in range (0, len(allDocuments)):
        if term.lower() in allDocuments[doc].lower().split():
            numDocumentsWithThisTerm = numDocumentsWithThisTerm + 1
 
    if numDocumentsWithThisTerm > 0:
        return 1.0 + math.log(float(len(allDocuments)) / numDocumentsWithThisTerm)
    else:
        return 1.0
    
def compute_idf(documents):
    idf_dict = {}
    for doc in documents:
        sentence = doc.split()
        for word in sentence:
            idf_dict[word] = inverseDocumentFrequency(word, documents)
    return idf_dict
idf_dict = compute_idf([doc1, doc2, doc3])

compute_idf([doc1, doc2, doc3])

{'I': 2.09861228866811,
 'want': 2.09861228866811,
 'to': 2.09861228866811,
 'start': 2.09861228866811,
 'learning': 1.4054651081081644,
 'charge': 2.09861228866811,
 'something': 1.4054651081081644,
 'in': 2.09861228866811,
 'life': 1.4054651081081644,
 'reading': 2.09861228866811,
 'about': 2.09861228866811,
 'no': 2.09861228866811,
 'one': 2.09861228866811,
 'else': 2.09861228866811,
 'knows': 2.09861228866811,
 'Never': 2.09861228866811,
 'stop': 2.09861228866811}

# 3.TF * IDF

Remember we are trying to find out relevant documents for the query: **life learning**

* For each term in the query multiply its normalized term frequency with its IDF on each document. 
* In Document1 for the term life the normalized term frequency is 0.1 and its IDF is 1.405465108. 
* Multiplying them together we get 0.140550715 (0.1 * 1.405465108). 
* 
Given below is TF * IDF calculations for life and learning in all the documents.

In [6]:
# tf-idf score across all docs for the query string("life learning")
def compute_tfidf_with_alldocs(documents , query):
    tf_idf = []
    index = 0
    query_tokens = query.split()
    df = pd.DataFrame(columns=['doc'] + query_tokens)
    for doc in documents:
        df['doc'] = np.arange(0 , len(documents))
        doc_num = tf_doc[index]
        sentence = doc.split()
        for word in sentence:
            for text in query_tokens:
                if(text == word):
                    idx = sentence.index(word)
                    tf_idf_score = doc_num[word] * idf_dict[word]
                    tf_idf.append(tf_idf_score)
                    df.iloc[index, df.columns.get_loc(word)] = tf_idf_score
        index += 1
    df.fillna(0 , axis=1, inplace=True)
    return tf_idf , df
            
documents = [doc1, doc2, doc3]
tf_idf , df = compute_tfidf_with_alldocs(documents , query)
print(df)

   doc      life  learning
0    0  0.140547  0.140547
1    1  0.175683  0.000000
2    2  0.000000  0.468488


# 4.Vector Space Models and Representation  – Cosine Similarity

The set of documents in a collection then is viewed as a set of vectors in a vector space. Each term will have its own axis. Using the formula given below we can find out the similarity between any two documents.

* > Cosine Similarity (d1, d2) =  Dot product(d1, d2) / ||d1|| * ||d2||
* > Dot product (d1,d2) = d1[0] * d2[0] + d1[1] * d2[1] * … * d1[n] * d2[n]
* > ||d1|| = square root(d1[0]2 + d1[1]2 + ... + d1[n]2)
* > ||d2|| = square root(d2[0]2 + d2[1]2 + ... + d2[n]2)


* Vectors deals only with numbers. In this example we are dealing with text documents. This was the reason why we used TF and IDF to convert text into numbers so that it can be represented by a vecto


The query entered by the user can also be represented as a vector. We will calculate the TF*IDF for the query

In [7]:
#Normalized TF for the query string("life learning")
def compute_query_tf(query):
    query_norm_tf = {}
    tokens = query.split()
    for word in tokens:
        query_norm_tf[word] = termFrequency(word , query)
    return query_norm_tf
query_norm_tf = compute_query_tf(query)
print(query_norm_tf)

{'life': 0.5, 'learning': 0.5}


In [8]:
#idf score for the query string("life learning")
def compute_query_idf(query):
    idf_dict_qry = {}
    sentence = query.split()
    documents = [doc1, doc2, doc3]
    for word in sentence:
        idf_dict_qry[word] = inverseDocumentFrequency(word ,documents)
    return idf_dict_qry
idf_dict_qry = compute_query_idf(query)
print(idf_dict_qry)

{'life': 1.4054651081081644, 'learning': 1.4054651081081644}


In [9]:
#tf-idf score for the query string("life learning")
def compute_query_tfidf(query):
    tfidf_dict_qry = {}
    sentence = query.split()
    for word in sentence:
        tfidf_dict_qry[word] = query_norm_tf[word] * idf_dict_qry[word]
    return tfidf_dict_qry
tfidf_dict_qry = compute_query_tfidf(query)
print(tfidf_dict_qry)

{'life': 0.7027325540540822, 'learning': 0.7027325540540822}


Let us now calculate the cosine similarity of the query and Document1.

Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

Dot product(Query, Document1) 
     = ((0.702753576) * (0.140550715) + (0.702753576)*(0.140550715))
     = 0.197545035151

||Query|| = sqrt((0.702753576)2 + (0.702753576)2) = 0.993843638185

||Document1|| = sqrt((0.140550715)2 + (0.140550715)2) = 0.198768727354

Cosine Similarity(Query, Document) = 0.197545035151 / (0.993843638185) * (0.198768727354)
                                        = 0.197545035151 / 0.197545035151
                                        = 1

In [12]:
#Cosine Similarity(Query,Document1) = Dot product(Query, Document1) / ||Query|| * ||Document1||

"""
Example : Dot roduct(Query, Document1) 

     life:
     = tfidf(life w.r.t query) * tfidf(life w.r.t Document1) +  / 
     sqrt(tfidf(life w.r.t query)) * 
     sqrt(tfidf(life w.r.t doc1))
     
     learning:
     =tfidf(learning w.r.t query) * tfidf(learning w.r.t Document1)/
     sqrt(tfidf(learning w.r.t query)) * 
     sqrt(tfidf(learning w.r.t doc1))

"""
def cosine_similarity(tfidf_dict_qry, df , query , doc_num):
    dot_product = 0
    qry_mod = 0
    doc_mod = 0
    tokens = query.split()
   
    for keyword in tokens:
        dot_product += tfidf_dict_qry[keyword] * df[keyword][df['doc'] == doc_num]
        #||Query||
        qry_mod += tfidf_dict_qry[keyword] * tfidf_dict_qry[keyword]
        #||Document||
        doc_mod += df[keyword][df['doc'] == doc_num] * df[keyword][df['doc'] == doc_num]
    qry_mod = np.sqrt(qry_mod)
    doc_mod = np.sqrt(doc_mod)
    #implement formula
    denominator = qry_mod * doc_mod
    cos_sim = dot_product/denominator
     
    return cos_sim

from collections import Iterable
def flatten(lis):
     for item in lis:
        if isinstance(item, Iterable) and not isinstance(item, str):
             for x in flatten(item):
                yield x
        else:        
             yield item


In [13]:
def rank_similarity_docs(data):
    cos_sim =[]
    for doc_num in range(0 , len(data)):
        cos_sim.append(cosine_similarity(tfidf_dict_qry, df , query , doc_num).tolist())
    return cos_sim
similarity_docs = rank_similarity_docs(documents)
doc_names = ["Document1", "Document2", "Document3"]
print(doc_names)
print(list(flatten(similarity_docs)))

['Document1', 'Document2', 'Document3']
[1.0, 0.7071067811865475, 0.7071067811865475]
